In [1]:
# Import the 'random' module to shuffle cards randomly
import random

# Define the suits, ranks, and values for the cards
suits = ["Hearts", "Diamonds", "Clubs", "Spades"]
ranks = ["Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine", "Ten", "Jack", "Queen", "King", "Ace"]
values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 11]

# Function to deal a random card
def deal_card():
    suit = random.choice(suits)
    rank = random.choice(ranks)
    value = values[ranks.index(rank)]
    return (rank, suit, value)

# Function to calculate the total value of a hand, considering aces
def calculate_hand_value(hand):
    value = sum(card[2] for card in hand)
    num_aces = sum(1 for card in hand if card[0] == "Ace")

    while value > 21 and num_aces:
        value -= 10
        num_aces -= 1

    return value

# Function to display player's and dealer's hands
def display_hands(player_hands, dealer_hand, show_all=False):
    for i, hand in enumerate(player_hands):
        print(f"\nPlayer's Hand {i + 1}:")
        for card in hand:
            print(f"{card[0]} of {card[1]}")

        player_total = calculate_hand_value(hand)
        print(f"Player Total: {player_total}")

    print("\nDealer's Hand:")
    if show_all:
        for card in dealer_hand:
            print(f"{card[0]} of {card[1]}")

        dealer_total = calculate_hand_value(dealer_hand)
        print(f"Dealer Total: {dealer_total}")
    else:
        print(f"{dealer_hand[0][0]} of {dealer_hand[0][1]}")
        print("Hidden Card")

# Function to play a single round
def play_round(balance):
    print(f"\nYour current balance: ${balance}")
    
    while True:
        bet = int(input("How much would you like to bet? "))
        if bet > balance:
            print("You do not have enough funds, go hit the ATM!")
            play_again = input("Do you want to play another hand? (Y/N)").upper()
            if play_again == 'N':
                return False, balance
        else:
            break

    player_hand = [deal_card(), deal_card()]
    dealer_hand = [deal_card(), deal_card()]

    while True:
        display_hands([player_hand], dealer_hand)

        player_value = calculate_hand_value(player_hand)
        dealer_value = calculate_hand_value(dealer_hand)

        if player_value == 21 and len(player_hand) == 2:  
            print(f"Blackjack! You win ${bet * 2.5}!")
            return True, balance + bet * 2.5

        action = input("\nDo you want to Hit (H), Stand (S), Double Down (D), or Split (P)? ").upper()

        if action == 'H':
            player_hand.append(deal_card())
        elif action == 'S':
            while dealer_value < 17:
                dealer_hand.append(deal_card())
                dealer_value = calculate_hand_value(dealer_hand)

            display_hands([player_hand], dealer_hand, show_all=True)

            if dealer_value > 21:
                print(f"Dealer busted! You win ${bet}! Your new balance: ${balance + bet}")
                return True, balance + bet
            elif player_value > dealer_value:
                print(f"You win ${bet}! Your new balance: ${balance + bet}")
                return True, balance + bet
            elif dealer_value > player_value:
                print(f"Dealer wins. Your new balance: ${balance - bet}")
                return True, balance - bet
            else:
                print(f"It's a tie! Your balance remains ${balance}.")
                return True, balance
        elif action == 'D':
            if bet * 2 > balance:
                print("You may not do this option as your bankroll is too low.")
            else:
                bet *= 2
                player_hand.append(deal_card())
                player_value = calculate_hand_value(player_hand)
                display_hands([player_hand], dealer_hand)

                while dealer_value < 17:
                    dealer_hand.append(deal_card())
                    dealer_value = calculate_hand_value(dealer_hand)

                display_hands([player_hand], dealer_hand, show_all=True)

                if player_value > 21:
                    print(f"Bust! You lose ${bet}! Your new balance: ${balance - bet}")
                    return True, balance - bet
                elif dealer_value > 21 or player_value > dealer_value:
                    print(f"You win ${bet}! Your new balance: ${balance + bet}")
                    return True, balance + bet
                elif dealer_value > player_value:
                    print(f"Dealer wins. Your new balance: ${balance - bet}")
                    return True, balance - bet
                else:
                    print(f"It's a tie! Your balance remains ${balance}.")
                    return True, balance
        elif action == 'P':
            if player_hand[0][2] == player_hand[1][2]:
                
                hand1 = [player_hand[0], deal_card()]
                hand2 = [player_hand[1], deal_card()]
                player_hands = [hand1, hand2]
                round_result, new_balance = play_round_with_hands(player_hands, dealer_hand, bet, balance)
                if round_result:
                    return True, new_balance
                else:
                    return False, balance
            else:
                print("Invalid action. You can only split when the cards have the same numerical value.")
        else:
            print("Invalid action. Please choose Hit (H), Stand (S), Double Down (D), or Split (P).")

        player_value = calculate_hand_value(player_hand)
        if player_value > 21:
            print(f"Bust! Dealer wins. Your new balance: ${balance - bet}")
            return True, balance - bet

# Function to play a round with multiple hands in case of splitting
def play_round_with_hands(player_hands, dealer_hand, bet, balance):
    while True:
        all_busted = True
        for i, hand in enumerate(player_hands):
            display_hands([hand], dealer_hand)

            player_value = calculate_hand_value(hand)
            dealer_value = calculate_hand_value(dealer_hand)

            if player_value == 21 and len(hand) == 2:
                print(f"Blackjack! Hand {i + 1} wins ${bet * 2.5}!")
                return True, balance + bet * 2.5

            action = input(f"\nDo you want to Hit (H), Stand (S), Double Down (D), or Split (P) for Hand {i + 1}? ").upper()

            if action == 'H':
                hand.append(deal_card())
                all_busted = False
            elif action == 'S':
                all_busted = False
            elif action == 'D':
                if bet * 2 > balance:
                    print("You may not do this option as your bankroll is too low.")
                else:
                    bet *= 2
                    hand.append(deal_card())
                    all_busted = False
            elif action == 'P':
                if hand[0][2] == hand[1][2]:
                    new_hand = [hand[1], deal_card()]
                    player_hands.append(new_hand)
                    hand[1] = deal_card()
                    all_busted = False
                else:
                    print("Invalid action. You can only split when the cards have the same numerical value.")
            else:
                print("Invalid action. Please choose Hit (H), Stand (S), Double Down (D), or Split (P) again.")

        if all_busted:
            print(f"All hands busted! Dealer wins. Your new balance: ${balance - bet}")
            return True, balance - bet

        while dealer_value < 17:
            dealer_hand.append(deal_card())
            dealer_value = calculate_hand_value(dealer_hand)

        display_hands(player_hands, dealer_hand, show_all=True)

        for i, hand in enumerate(player_hands):
            player_value = calculate_hand_value(hand)

            if dealer_value > 21 or player_value > dealer_value:
                print(f"Hand {i + 1} wins ${bet}! Your new balance: ${balance + bet}")
                return True, balance + bet
            elif dealer_value > player_value:
                print(f"Hand {i + 1} loses. Your new balance: ${balance - bet}")
                return True, balance - bet
            else:
                print(f"Hand {i + 1} ties with the dealer. Your balance remains ${balance}.")

# Initial balance for the player
balance = 100

# Main game loop
while True:
    round_result, balance = play_round(balance)
    if not round_result:
        break
    play_again = input("Do you want to play another round? (Y/N)").upper()
    if play_again != 'Y':
        break


Your current balance: $100
How much would you like to bet? 24

Player's Hand 1:
Eight of Spades
Two of Hearts
Player Total: 10

Dealer's Hand:
Four of Spades
Hidden Card

Do you want to Hit (H), Stand (S), Double Down (D), or Split (P)? D

Player's Hand 1:
Eight of Spades
Two of Hearts
Five of Hearts
Player Total: 15

Dealer's Hand:
Four of Spades
Hidden Card

Player's Hand 1:
Eight of Spades
Two of Hearts
Five of Hearts
Player Total: 15

Dealer's Hand:
Four of Spades
Queen of Diamonds
Five of Clubs
Dealer Total: 19
Dealer wins. Your new balance: $52
Do you want to play another round? (Y/N)N
